In [31]:
import pandas as pd
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [32]:
%sql sqlite:///../datadir10000/crawl-data.sqlite

'Connected: @../datadir10000/crawl-data.sqlite'

In [33]:
# %%sql 
# SELECT *
# FROM javascript
# WHERE document_url = 'https://fingerprint-react-demo.vercel.app/imprintjs'
# LIMIT 100

In [34]:
%%sql 
SELECT DISTINCT symbol 
FROM javascript

 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../datadirDemoSites/crawl-data.sqlite
Done.


symbol
window.navigator.userAgent
window.document.cookie
window.localStorage
Storage.getItem
window.navigator.product
Storage.setItem
window.sessionStorage
window.navigator.appName
window.document.referrer
window.navigator.onLine


In [35]:
import ast

def Canvas(df :  pd.DataFrame) -> bool:
    heightRec = False
    widthRec = False
    colors = set()
    charecters = set()
    Extraction = False
    for row in df.itertuples():
        match row.symbol:
            case 'HTMLCanvasElement.height':
                if row.operation == 'set' and row.value and float(row.value) >= 16:
                    heightRec = True
            case 'HTMLCanvasElement.width':
                if row.operation == 'set' and row.value and float(row.value) >= 16:
                    widthRec = True
            case 'CanvasRenderingContext2D.fillText':
                args = ast.literal_eval(row.arguments)
                for char in args[0]:
                    charecters.add(char)
            case 'CanvasRenderingContext2D.fillStyle':
                if row.operation == 'set' and row.value:
                    colors.add(row.value)
            case 'HTMLCanvasElement.toDataURL':
                Extraction = True
            case 'CanvasRenderingContext2D.getImageData':
                Extraction = True
    return heightRec and widthRec and ( len(colors) > 2 or len(charecters) > 10) and Extraction

# functions = [ ("Canvas", Canvas ) ]

In [36]:
script_urls = %sql SELECT DISTINCT script_url, top_level_url FROM javascript LIMIT 500
canvasFingerprinters = []
for row in script_urls:
    script : str = row[0]
    top_level_url : str = row[1]
    result = %sql SELECT * FROM javascript WHERE script_url = '{script}' AND top_level_url = '{top_level_url}'
    if result:
        df : pd.DataFrame = result.DataFrame()
        if Canvas(df):
            canvasFingerprinters.append({ "script": script, "top_level_url" : top_level_url})
    

 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../datadirDemoSites/crawl-data.sqlite
Done.
 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../datadirDemoSites/crawl-data.sqlite
Done.
 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../datadirDemoSites/crawl-data.sqlite
Done.
 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../datadirDemoSites/crawl-data.sqlite
Done.
 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../datadirDemoSites/crawl-data.sqlite
Done.
 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../datadirDemoSites/crawl-data.sqlite
Done.
 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../datadirDemoSites/crawl-data.sqlite
Done.
 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../datadirDemoSites/crawl-data.sqlite
Done.
 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../datadirDemoSites/crawl-data.sqlite
Done.
 * sqlite:///../datadir10000/crawl-data.sqlite
   sqlite:///../d

In [37]:
import json
print( json.dumps( canvasFingerprinters, indent= 4 ) )

[
    {
        "script": "https://www.digikey.at/L6e32Mz9A9AC/gDO9lTSx9Q/I0/u13t8mrt1z/Ty5sDFsCOg/EGhS/DT0KITgB",
        "top_level_url": "https://www.digikey.at/"
    },
    {
        "script": "https://cdn.sift.com/s.js",
        "top_level_url": "https://www.digikey.at/"
    }
]
